In [281]:
from sklearn import linear_model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
x_tr=pd.read_csv('../X_train.csv', sep=',')
y_tr=pd.read_csv('../y_train.csv', sep=',')
x_te=pd.read_csv('../X_test.csv', sep=',')

# Initial empty prediction dataframe
index = np.arange(len(x_te.iloc[:,0]))
columns = ["Id"]
rows = len(x_te.iloc[:,0])
data = np.arange(rows, dtype=int)
prediction = pd.DataFrame(data, columns=columns)

In [282]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer
# Scale data MinMax
def scale(type="minmax", x_train=x_tr, x_test=x_te):
    if type=="minmax":
        scaler_x = MinMaxScaler()
    elif type=="standard":
        scaler_x = StandardScaler()
        
    scaler_x = scaler_x.fit(x_train)  # Don't cheat - fit only on training data
    x_train = scaler_x.transform(x_train)
    x_test = scaler_x.transform(x_test)  # apply same transformation to test data
    
    return x_train, x_test, scaler_x

"""
print(x_train.mean(axis = 0))
print(x_test.mean(axis = 0))
print(y_train.mean(axis = 0))
print(x_train.std(axis = 0))
print(x_test.std(axis = 0))
print(y_train.std(axis = 0))
"""

'\nprint(x_train.mean(axis = 0))\nprint(x_test.mean(axis = 0))\nprint(y_train.mean(axis = 0))\nprint(x_train.std(axis = 0))\nprint(x_test.std(axis = 0))\nprint(y_train.std(axis = 0))\n'

In [283]:
# Polynomial features
from sklearn.preprocessing import PolynomialFeatures
def poly(features=3, x_train=x_tr, x_test=x_te):

    poly = PolynomialFeatures(features)

    x_train = poly.fit_transform(x_train)
    x_test = poly.fit_transform(x_test)
    
    return x_train, x_test

In [334]:
#makescorer
from sklearn.metrics.scorer import make_scorer
def rmse(true, pred):
    mse = mean_squared_error(true, pred)
    rmse = sqrt(mse)

    print(rmse)
    return rmse

In [338]:
#Lasso regression pipeline (l1 regularisation)

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipe = Pipeline([("scaler1", StandardScaler()),
                  ("poly", PolynomialFeatures(degree=5)),
                  ("scaler2", StandardScaler()),
                  ("lasso", linear_model.ElasticNetCV(n_alphas=1, cv=5, l1_ratio=1, max_iter=1e6))
                 ])
param_grid = {
    'lasso__alphas': list(np.linspace(1, 10, 10).reshape(10,1))
}
scorer = make_scorer(rmse, greater_is_better=False)
search = GridSearchCV(pipe, param_grid, cv=5, error_score="raise", scoring=scorer)
print("Best Params:" + str(search.fit(x_tr, y_tr["PRP"]).best_params_))
model = search.fit(x_tr, y_tr["PRP"]).best_estimator_
prediction["PRP"] = np.around(model.predict(x_te)).astype(int)
#prediction.iloc[35]["PRP"] *= 2
print(prediction.to_string(index=False))
prediction.to_csv("n_lr_results.csv", index=False)
print(model.score(x_tr, y_tr["PRP"]))
calc_rmse()

86.20390202389234
35.41675428840772
73.18881872809477
890.0689008890522
98.14407259656686
156.07182222591916
32.58849903269546
75.11068443858872
661.4897705472122
366.92164100120374
190.82114147812797
30.72630930050408
64.07694873225365
405.2541214708205
125.58706509639731
202.5731109467631
29.9483054300711
58.42925890255348
175.19901381060356
84.60056297000621
207.85212536864228
29.645793033279382
58.659973170298656
37.020481534356875
83.43357806139709
214.04834475534838
29.161012595316432
54.9352319151283
50.572815935168656
84.45325101599137
220.32995793247366
28.68748399768191
52.57683262099288
49.8769553381917
85.08270275252711
226.11071715094715
28.69409186508266
53.021430482697966
49.20927929702532
85.76098900004185
231.98902485684764
28.729817011945048
53.18587386170484
48.590944207339966
86.46945097943424
239.6723748137407
28.82752219908417
53.306289692894374
48.028411994680106
87.20749623043258


C:\Users\maxkb\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Params:{'lasso__alphas': array([5.])}
86.20390202389234
35.41675428840772
73.18881872809477
890.0689008890522
98.14407259656686
156.07182222591916
32.58849903269546
75.11068443858872
661.4897705472122
366.92164100120374
190.82114147812797
30.72630930050408
64.07694873225365
405.2541214708205
125.58706509639731
202.5731109467631
29.9483054300711
58.42925890255348
175.19901381060356
84.60056297000621
207.85212536864228
29.645793033279382
58.659973170298656
37.020481534356875
83.43357806139709
214.04834475534838
29.161012595316432
54.9352319151283
50.572815935168656
84.45325101599137
220.32995793247366
28.68748399768191
52.57683262099288
49.8769553381917
85.08270275252711
226.11071715094715
28.69409186508266
53.021430482697966
49.20927929702532
85.76098900004185
231.98902485684764
28.729817011945048
53.18587386170484
48.590944207339966
86.46945097943424
239.6723748137407
28.82752219908417
53.306289692894374
48.028411994680106
87.20749623043258


C:\Users\maxkb\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


 Id   PRP
  0    44
  1   154
  2    21
  3   387
  4    55
  5    24
  6    73
  7    32
  8    41
  9    25
 10    33
 11    26
 12   263
 13    48
 14   149
 15    35
 16    36
 17    46
 18    35
 19   152
 20    97
 21    29
 22    65
 23    22
 24   154
 25   116
 26    75
 27   381
 28    30
 29   300
 30   114
 31    16
 32    71
 33   178
 34    43
 35  1011
 36    24
 37   381
 38    43
 39    36
 40   114
0.9433688763934911
41.10159928049801


41.10159928049801

In [347]:
#Lasso regression pipeline (l1 regularisation)

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipe = Pipeline([("scaler1", StandardScaler()),
                  ("poly", PolynomialFeatures(degree=2)),
                  ("scaler2", StandardScaler()),
                  ("lasso", linear_model.LassoLarsCV(cv=5, max_iter=1e6))
                 ])

model = pipe.fit(x_tr, y_tr["PRP"])
prediction["PRP"] = np.around(model.predict(x_te)).astype(int)
#prediction.iloc[35]["PRP"] *= 2
print(prediction.to_string(index=False))
prediction.to_csv("n_lr_results.csv", index=False)
print(model.score(x_tr, y_tr["PRP"]))
calc_rmse()

 Id  PRP
  0   32
  1  154
  2   36
  3  311
  4   57
  5   36
  6   76
  7   36
  8   44
  9   32
 10    7
 11   36
 12  254
 13   53
 14  151
 15   42
 16   44
 17   49
 18   37
 19  162
 20   93
 21   34
 22   66
 23   37
 24  154
 25  114
 26   78
 27  365
 28   36
 29  257
 30  113
 31   33
 32   71
 33  176
 34   44
 35  779
 36   37
 37  365
 38   46
 39   40
 40  113
0.9237792728726125
65.48580184973258


65.48580184973258

In [127]:
# Ridge Regression
x_train, x_test, scaler = scale(type="minmax")
x_train, x_test = poly(features=3, x_train=x_train, x_test=x_test)

x_train, x_test, scaler2 = scale(type="minmax", x_train=x_train, x_test=x_test)

reg = linear_model.LassoCV(cv=5, max_iter=1e6)
reg.fit(x_train, y_tr["PRP"])
print(reg.score(x_train, y_tr["PRP"]))

rr = pd.DataFrame(data, columns=columns)
rr["PRP"] = reg.predict(x_test)
prp = np.around(rr["PRP"]).astype(int)
prediction["PRP"] = prp
    
print(prediction.to_string(index=False))
prediction.to_csv("n_lr_results.csv", index=False)

calc_rmse()

0.9535201711153846
 Id  PRP
  0   42
  1  149
  2   23
  3  382
  4   51
  5   25
  6   68
  7   29
  8   39
  9   22
 10   31
 11   25
 12  270
 13   41
 14  152
 15   31
 16   34
 17   40
 18   28
 19  149
 20   53
 21   25
 22   59
 23   24
 24  152
 25  122
 26   75
 27  412
 28   27
 29  462
 30  120
 31   21
 32   61
 33  170
 34   40
 35  962
 36   21
 37  412
 38   41
 39   32
 40  120
60.08936434442516


60.08936434442516

In [285]:
# Get real PRP values
real_values=pd.read_csv('../machine.csv', sep=',')
raw_test=pd.read_csv('../X_test.csv', sep=',')
columns = ["MYCT", "MMIN", "MMAX", "CACH", "CHMIN", "CHMAX", "PRP", "ERP"]
index = np.arange(len(raw_test))
final = pd.DataFrame(index=index, columns=columns)

for i in range(len(x_test)):
    row = raw_test.loc[i]
    MYCT = row["MYCT"] == real_values["MYCT"]
    MMIN = row["MMIN"] == real_values["MMIN"]
    MMAX = row["MMAX"] == real_values["MMAX"]
    CACH = row["CACH"] == real_values["CACH"]
    CHMIN = row["CHMIN"] == real_values["CHMIN"]
    CHMAX = row["CHMAX"] == real_values["CHMAX"]
    
    new_final_row = real_values[MYCT&MMIN&MMAX&CACH&CHMIN&CHMAX]
    if len(final) > 1:
        new_final_row = new_final_row.iloc[0]
    final.iloc[i] = new_final_row
print(final["PRP"])
#print(final["ERP"])

0       38
1      208
2       12
3      307
4       45
5       45
6       52
7       20
8       53
9       25
10      30
11      38
12     248
13      71
14     214
15      36
16      32
17      36
18      31
19     134
20      41
21       8
22     105
23      24
24     106
25     133
26      45
27     367
28       6
29     198
30      51
31      20
32     109
33     237
34      30
35    1150
36      18
37     367
38      21
39      42
40      51
Name: PRP, dtype: int64


In [286]:
# Calculate MSE

from sklearn.metrics import mean_squared_error
from math import sqrt

def calc_rmse():
    slice = 12
    mse = mean_squared_error(final.iloc[:]["PRP"], prediction.iloc[:]["PRP"])
    rmse = sqrt(mse)

    print(rmse)
    return rmse
calc_rmse()

KeyError: 'PRP'